In [ ]:
!nvidia-smi

Sat Jul  8 10:27:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Loading deep learning algorithm
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import keras
#import keras.backend as K
#from keras.models import Sequential
#from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

IMAGE_SIZE = [224,224]
CLASS=4
inception = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#model = VGG16(weights='imagenet', include_top=False)
for layer in inception.layers:
    layer.trainable = False
#folders = glob('C:\rafid\guava disease research\k_guava\train_image/*')
x = Flatten()(inception.output)
prediction = Dense(CLASS, activation='softmax')(x)
model = Model(inputs=inception.input, outputs=prediction)
adam = keras.optimizers.Adam(lr = 0.001)
model.compile(
  loss='categorical_crossentropy',
  optimizer = adam,
  metrics=['accuracy']
)
print("\n\n")
model.summary()

58889256/58889256 [==============================] - 4s 0us/step





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/defense/Split-dataset/train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/defense/Split-dataset/val',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/defense/Split-dataset/test',
                                            target_size = (224,224),
                                            batch_size = 1,
                                            class_mode = 'categorical')
print("\n\n")
model.optimizer.get_config()

In [ ]:
filepath = "/content/drive/MyDrive/defense/Accuracy/vgg16/highest_val2.h5"
filepath2 = "/content/drive/MyDrive/defense/Accuracy/vgg16/highest_train2.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='accuracy', verbose=1,
                             save_best_only=True, mode='max')
callbacks_list = [checkpoint1,checkpoint2]
r = model.fit_generator(
    training_set,
    epochs=160,
    validation_data=val_set,
    steps_per_epoch=len(training_set),
    validation_steps=len(val_set),
    callbacks=callbacks_list
)
model.save_weights("/content/drive/MyDrive/defense/Accuracy/vgg166/end2.h5")

In [ ]:
#plot of accuracy and loss

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.title('Training and validation Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
#evaluating the model (test acc)
#batch size = 32
model.load_weights('/content/drive/MyDrive/Onion Diseases/Accuracy /VGG16/highest_val2.h5')
preds = model.evaluate_generator(test_set)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Loss = 0.6372169852256775
Test Accuracy = 0.8952381014823914


In [ ]:
#confusion matrix

#you have to set test bath size=1 before running the cell
import pandas as pd
import seaborn as sn
import tensorflow as tf
model.load_weights('/content/drive/MyDrive/Onion Diseases/Accuracy /VGG16/highest_val2.h5')
filenames=test_set.filenames
nb_samples=len(test_set)
y_prob=[]
y_act=[]
test_set.reset()
for _ in range (nb_samples):
    X_test,Y_test = test_set.next()
    y_prob.append(model.predict(X_test))
    y_act.append(Y_test)
predicted_class=[list(training_set.class_indices.keys())[i.argmax()] for i in y_prob]
actual_class=[list(training_set.class_indices.keys())[i.argmax()]for i in y_act]
out_df=pd.DataFrame(np.vstack([predicted_class,actual_class]).T,columns=['predicted_class','actual_class'])
confusion_matrix=pd.crosstab(out_df['actual_class'],out_df['predicted_class'],rownames=['Actual'],colnames=['Predicted'])
import matplotlib.pyplot as plt
sn.heatmap(confusion_matrix,cmap='flare', annot=True, fmt='d')
plt.show()
#plt.savefig('/content/drive/MyDrive/model weights/vgg16_AugGfb_split1_maxval_3')

In [ ]:
lst=[r.history['loss'],r.history['val_loss'],r.history['accuracy'],r.history['val_accuracy']]

In [ ]:
import pandas as pd
df=pd.DataFrame(lst)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159
0,1.063946,0.562295,0.419263,0.332225,0.281066,0.250044,0.227190,0.205565,0.164640,0.161166,0.150859,0.135544,0.137505,0.127168,0.121686,0.103266,0.081665,0.113968,0.083766,0.086503,0.071566,0.068733,0.065447,0.072682,0.053664,0.074712,0.053769,0.058828,0.066965,0.055327,0.066766,0.062500,0.057807,0.063958,0.060098,0.058977,0.057235,0.049694,0.056211,0.037851,...,0.034618,0.025073,0.037075,0.024423,0.032857,0.025307,0.037053,0.021943,0.048483,0.052883,0.036644,0.025488,0.026418,0.017771,0.024358,0.028515,0.040957,0.037452,0.033422,0.019166,0.022417,0.029063,0.013146,0.028047,0.024241,0.017351,0.031081,0.040792,0.025646,0.046916,0.042484,0.024934,0.023529,0.017365,0.019025,0.036077,0.036043,0.022657,0.025956,0.054078
1,0.744979,0.635575,0.424732,0.472201,0.387601,0.390967,0.411083,0.355824,0.317913,0.318755,0.324523,0.389066,0.313781,0.337654,0.329102,0.318284,0.392043,0.310019,0.308675,0.318253,0.310531,0.399959,0.301473,0.370622,0.336134,0.301352,0.342349,0.309791,0.324641,0.318276,0.378463,0.363966,0.370685,0.359754,0.444659,0.367711,0.449735,0.381284,0.357317,0.379763,...,0.483296,0.479123,0.674842,0.523996,0.478137,0.472133,0.528279,0.665578,0.725088,0.707837,0.535278,0.653428,0.550126,0.530339,0.666191,0.774262,0.783743,0.787930,0.497952,0.552518,0.590774,0.530985,0.605204,0.754556,0.571858,0.571094,0.573701,0.627058,0.589159,0.625886,0.552360,0.704162,0.759786,0.665776,0.680469,0.590573,0.577251,0.520232,0.625593,0.676441
2,0.575404,0.789763,0.859842,0.892477,0.912058,0.921333,0.926829,0.935761,0.953624,0.953281,0.954998,0.962899,0.954655,0.960838,0.964273,0.970457,0.982824,0.965991,0.976984,0.976297,0.981106,0.984541,0.982824,0.981106,0.987290,0.980419,0.987977,0.984885,0.982137,0.984885,0.984541,0.980076,0.983854,0.981450,0.979732,0.980076,0.986603,0.986946,0.985572,0.989694,...,0.989694,0.992442,0.989007,0.992786,0.990381,0.992442,0.990381,0.992442,0.986259,0.983511,0.989007,0.992442,0.993817,0.994160,0.995534,0.990381,0.989694,0.987977,0.991068,0.995534,0.994160,0.993817,0.995191,0.992442,0.992786,0.994847,0.989694,0.989694,0.995191,0.986603,0.987977,0.993129,0.992442,0.994847,0.994504,0.990381,0.988320,0.993473,0.993129,0.983511
3,0.711191,0.778580,0.849579,0.820698,0.858002,0.851986,0.847172,0.876053,0.877256,0.888087,0.883273,0.876053,0.885680,0.889290,0.896510,0.904934,0.866426,0.896510,0.889290,0.882070,0.895307,0.889290,0.896510,0.873646,0.897714,0.897714,0.890493,0.909747,0.894103,0.897714,0.878460,0.888087,0.878460,0.883273,0.879663,0.892900,0.873646,0.886883,0.890493,0.892900,...,0.902527,0.900120,0.866426,0.897714,0.900120,0.906137,0.894103,0.882070,0.884477,0.886883,0.892900,0.885680,0.889290,0.906137,0.883273,0.867629,0.865223,0.876053,0.910951,0.892900,0.889290,0.903730,0.886883,0.879663,0.888087,0.888087,0.900120,0.888087,0.885680,0.889290,0.892900,0.886883,0.876053,0.884477,0.886883,0.894103,0.890493,0.902527,0.884477,0.891697


In [ ]:
df.to_csv('IRNadam.csv')